In [1]:
import os 
os.chdir("../")
%pwd

'/home/thnhan/Documents/GIT/ireuron_end2end/wine-quality-prediction'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig():
    root_dir: Path
    data_path: Path

In [3]:
from wine_quality_project.constants import *
from wine_quality_project.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation(self)-> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config


In [9]:
from wine_quality_project import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [17]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-04-04 23:08:56,244: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-04 23:08:56,247: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-04 23:08:56,250: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-04 23:08:56,251: INFO: common: created directory at: artifacts]
[2024-04-04 23:08:56,251: INFO: common: created directory at: artifacts/data_transformation]
[2024-04-04 23:08:56,268: INFO: 1383578707: Splitted data into training and test sets]
[2024-04-04 23:08:56,269: INFO: 1383578707: (1279, 12)]
[2024-04-04 23:08:56,269: INFO: 1383578707: (320, 12)]
